### 선수 개인정보 데이터 크롤러 제작
- KOKO 현대캐피탈 스카이워커스 페이지 https://www.kovo.co.kr/team/21131_player_list.asp?t_code=1005&s_part=1 에서 각 선수별 링크 스크래핑 후
- 각 선수 링크로 들어가 선수번호, 포지션, 이미지 링크, 이름, 생년월일 등 선수정보 스크래핑 후
- 위 정보를 담은 데이터프레임 생성

In [17]:
import requests
import re
import warnings

# 불필요한 경고 출력을 방지합니다.
warnings.filterwarnings('ignore')


In [18]:
url = "https://www.kovo.co.kr/team/21131_player_list.asp?t_code=1005&s_part=1"
html = requests.get(url, verify = False).text

In [19]:
from bs4 import BeautifulSoup as bs
soup = bs(html,"lxml")

In [20]:
# 선수정보 탭만 가져오기
player_all = soup.select("#tab2 > ul > li ")

In [21]:
# link_list에 각 선수별 링크 저장
link_list = []

for i in player_all :
    linkfile = "https://www.kovo.co.kr/team/" + i.find("a").get("href")
    link_list.append(linkfile)
print(link_list)



['https://www.kovo.co.kr/team/21132_player_view.asp?p_code=1000229&t_code=1005&s_part=1', 'https://www.kovo.co.kr/team/21132_player_view.asp?p_code=1000059&t_code=1005&s_part=1', 'https://www.kovo.co.kr/team/21132_player_view.asp?p_code=0001178&t_code=1005&s_part=1', 'https://www.kovo.co.kr/team/21132_player_view.asp?p_code=0001377&t_code=1005&s_part=1', 'https://www.kovo.co.kr/team/21132_player_view.asp?p_code=0001365&t_code=1005&s_part=1', 'https://www.kovo.co.kr/team/21132_player_view.asp?p_code=0001462&t_code=1005&s_part=1', 'https://www.kovo.co.kr/team/21132_player_view.asp?p_code=0001967&t_code=1005&s_part=1', 'https://www.kovo.co.kr/team/21132_player_view.asp?p_code=1000138&t_code=1005&s_part=1', 'https://www.kovo.co.kr/team/21132_player_view.asp?p_code=0000853&t_code=1005&s_part=1', 'https://www.kovo.co.kr/team/21132_player_view.asp?p_code=0002255&t_code=1005&s_part=1', 'https://www.kovo.co.kr/team/21132_player_view.asp?p_code=0001873&t_code=1005&s_part=1', 'https://www.kovo.co

In [25]:
link_list.append("https://www.kovo.co.kr/team/22002_player_view.asp?s_part=1&p_code=0001940")
link_list.append("https://www.kovo.co.kr/team/22002_player_view.asp?s_part=1&p_code=1000176")

In [26]:
player_number = []
player_position = []

player_name = []
player_birth = []
player_height = []
player_weight = []
player_history = []
player_school = []
player_img = []

for player_link in link_list :
    url = player_link
    player_html = requests.get(url, verify = False).text
    player_soup = bs(player_html,"lxml")

    no = player_soup.find("p",{"class":"no"}).get_text()

    # 선수 번호
    player_number.append(no.split("\xa0")[0])

    # 선수 포지션
    player_position.append(no.split("\xa0")[1])

    # 선수 이미지 링크
    player_img.append("https://www.kovo.co.kr/"+player_soup.select("div.clearfix > div.img > img")[0].get("src"))

    # 선수 이름
    player_name.append(str(player_soup.select("div.info > p.name")).strip('[<p class="name">'"</p>]"))

    # 선수 생년월일
    player_birth.append(str(player_soup.select("dl.clearfix > dd")[0]).strip("<dd>""</dd>"))

    # 선수 신장
    player_height.append(str(player_soup.select("dl.clearfix > dd")[1]).split("/")[0].strip("<dd>"" "))

    # 선수 몸무게
    player_weight.append(str(player_soup.select("dl.clearfix > dd")[1]).split("/")[1].strip("<"" "))

    # 선수 출신학교
    player_school.append(str(player_soup.select("dl.clearfix > dd")[2]).strip("<dd>""</dd>"))

    # 선수 선수 이력
    player_history.append(re.sub("\xa0", "", player_soup.select("dl.clearfix > dd")[-1].get_text().strip("\n")))

    # 선수 선수 이력 여러개일 경우 split으로 나눠줌
    # re.sub("\xa0", "", player_soup.select("dl.clearfix > dd")[-1].get_text().strip("\n")).split("\n")

In [27]:
import pandas as pd


In [28]:
player_df = pd.DataFrame({"player_name":player_name,"player_number":player_number,"player_position":player_position,"player_img":player_img,"player_birth":player_birth,"player_height":player_height,"player_weight":player_weight,"player_school":player_school,"player_history":player_history})


오레올 선수 출신 학교 정보 없음

In [31]:
player_df.iloc[4,7] = "NA"
player_df.iloc[24:,7] = "NA"

In [32]:
player_df

,player_name,player_number,player_position,player_img,player_birth,player_height,player_weight,player_school,player_history
0,고우진,NO.24,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/1000229...,2000년 04월 23일,190cm,85kg,부송초-남성중-남성고-한양대,2022-2023 V리그 현대캐피탈드래프트수련순위 (2022.10.19)
1,김선호,NO.6,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/1000059...,1999년 01월 18일,187cm,82kg,부송초-남성중-남성고-한양대,2020-2021 V리그 현대캐피탈드래프트1라운드 1순위 (2020.10.16)
2,박주형,NO.9,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/0001178...,1987년 08월 05일,193cm,79kg,대천초 / 성지중 / 성지공고 / 성균관대,2010-2011 V리그 우리캐피탈드래프트1라운드 2순위 (2010.11.04)\...
3,송준호,NO.8,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/0001377...,1991년 06월 05일,192cm,83kg,중앙중 / 중앙고 / 홍익대,2012-2013 V리그 현대캐피탈드래프트1라운드 4순위 (2012.10.22)\...
4,오레올,NO.10,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/0001365...,1986년 07월 22일,207cm,104kg,NA,2012-2013 V리그 LIG손해보험 (2012.10.01)\n2015-2016...
5,전광인,NO.12,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/0001462...,1991년 09월 18일,194cm,88kg,하동초 / 동명중·고 / 성균관대,2013-2014 V리그 한국전력드래프트1라운드 1순위 (2013.09.10)\n...
6,함형진,NO.4,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/0001967...,1995년 06월 25일,186cm,80kg,설악중 / 속초고 / 중부대,2017-2018 V리그 현대캐피탈드래프트2라운드 7순위 (2017.07.01)\...
7,홍동선,NO.18,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/1000138...,2001년 05월 16일,198cm,80kg,안산서초 / 송산중 / 송산고 / 인하대,2021-2022 V리그 현대캐피탈드래프트1라운드 1순위 (2021.10.06)
8,문성민,NO.15,OPPOSITE SPIKER,https://www.kovo.co.kr//upfiles/player/0000853...,1986년 09월 14일,198cm,90kg,부산명륜초 / 부산동성중 / 부산동성고 / 경기대,2008-2009 V리그 KEPCO45드래프트1라운드 1순위 (2008.11.04...
9,최은석,NO.33,OPPOSITE SPIKER,https://www.kovo.co.kr//upfiles/player/0002255...,1998년 03월 17일,193cm,75kg,대석초-필미중-제일고-중부대,2019-2020 V리그 현대캐피탈드래프트1라운드 7순위 (2019.10.31)\...


In [35]:
player_df.loc[26] = ["차영석","NO.1", "MIDDLE BLOCKER","https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAxODAyMTZfODYg%2FMDAxNTE4NzY5MDA3NTYw.UdQqc8bQLfo7wmgtFRW9HnmzjdV3bmAQOkjYOuImA3Eg.JXV3QXe8jEf3_vYb5ABgw7kcUZJaCLyZCBR1hndcVyYg.JPEG.jcg8708%2F0001866_1.jpg&type=sc960_832","1994년 04월 17일","193cm","85kg","담양동초-담양중-순천제일고-인하대","2016년 현대캐피탈 스카이워커스 입단"]
player_df.loc[27] = ["김형진","NO.4", "SETTER","https://w.namu.la/s/58d9b4c005f6e13ba2a7705806523d5a5465a93ca4ac69c569cbfc7ba60229f959e9aa23dd54ffa8ecfdf0a9bf5d2933fc0c432ebdf20736e59df4d9acc8d3ca40ba7758f2279ece3611bd91c1f53bf9c5c7b5ee5739912135f06813abf58acc","1995년 03월 10일","186cm","78kg","토평초-남성중-남성고-홍익대","2017년 대학배구 세터상 2018년 KOVO컵 라이징 스타상"]

In [36]:
player_df

,player_name,player_number,player_position,player_img,player_birth,player_height,player_weight,player_school,player_history
0,고우진,NO.24,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/1000229...,2000년 04월 23일,190cm,85kg,부송초-남성중-남성고-한양대,2022-2023 V리그 현대캐피탈드래프트수련순위 (2022.10.19)
1,김선호,NO.6,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/1000059...,1999년 01월 18일,187cm,82kg,부송초-남성중-남성고-한양대,2020-2021 V리그 현대캐피탈드래프트1라운드 1순위 (2020.10.16)
2,박주형,NO.9,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/0001178...,1987년 08월 05일,193cm,79kg,대천초 / 성지중 / 성지공고 / 성균관대,2010-2011 V리그 우리캐피탈드래프트1라운드 2순위 (2010.11.04)\...
3,송준호,NO.8,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/0001377...,1991년 06월 05일,192cm,83kg,중앙중 / 중앙고 / 홍익대,2012-2013 V리그 현대캐피탈드래프트1라운드 4순위 (2012.10.22)\...
4,오레올,NO.10,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/0001365...,1986년 07월 22일,207cm,104kg,NA,2012-2013 V리그 LIG손해보험 (2012.10.01)\n2015-2016...
5,전광인,NO.12,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/0001462...,1991년 09월 18일,194cm,88kg,하동초 / 동명중·고 / 성균관대,2013-2014 V리그 한국전력드래프트1라운드 1순위 (2013.09.10)\n...
6,함형진,NO.4,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/0001967...,1995년 06월 25일,186cm,80kg,설악중 / 속초고 / 중부대,2017-2018 V리그 현대캐피탈드래프트2라운드 7순위 (2017.07.01)\...
7,홍동선,NO.18,OUTSIDE HITTER,https://www.kovo.co.kr//upfiles/player/1000138...,2001년 05월 16일,198cm,80kg,안산서초 / 송산중 / 송산고 / 인하대,2021-2022 V리그 현대캐피탈드래프트1라운드 1순위 (2021.10.06)
8,문성민,NO.15,OPPOSITE SPIKER,https://www.kovo.co.kr//upfiles/player/0000853...,1986년 09월 14일,198cm,90kg,부산명륜초 / 부산동성중 / 부산동성고 / 경기대,2008-2009 V리그 KEPCO45드래프트1라운드 1순위 (2008.11.04...
9,최은석,NO.33,OPPOSITE SPIKER,https://www.kovo.co.kr//upfiles/player/0002255...,1998년 03월 17일,193cm,75kg,대석초-필미중-제일고-중부대,2019-2020 V리그 현대캐피탈드래프트1라운드 7순위 (2019.10.31)\...


In [37]:
player_df.to_excel("C:/kovo/KOVO_player_data.xlsx",index = False)